In [1]:
import numpy as np

In [2]:
actions = ['N','E','S','W']
states = [[a,b] for a in range(4) for b in range(4)]
rewards = [-1,0,10,5]

In [3]:
def prob(s_prime,r,s,a):
    
    # transport to (3, 1) and get reward 10 at (0, 1)
    if s == [0, 1]:
        if s_prime == [3, 1] and r == 10:
            return 1.0
        else:
            return 0.0
        
    # transport to (3, 3) and get reward 5 at (1, 3)    
    if s == [1, 3]:
        if s_prime == [3, 3] and r == 5:
            return 1.0
        else:
            return 0.0

    if a == 'N':

        if s == [1, 1]:
            if s_prime == [3, 1] and r == 10:
                return 1.0
            else:
                return 0.0
        
        elif s == [2, 3]:
            if s_prime == [3, 3] and r == 5:
                return 1.0
            else:
                return 0.0
            
        elif s[0] > 0 and s_prime == [s[0] - 1, s[1]] and r == 0:
            return 1.0
        
        # touch the wall
        elif s[0] == 0 and s_prime == s and r == -1:
            return 1.0
        
        else:
            return 0.0
        
    elif a == 'E':
        
        if s == [0, 0]:
            if s_prime == [3, 1] and r == 10:
                return 1.0
            else:
                return 0.0
            
        elif s == [1,2]:
            if s_prime == [3, 3] and r==5:
                return 1.0
            else: 
                return 0.0
            
        elif s[1] < 3 and s_prime == [s[0], s[1] + 1] and r == 0:
            return 1.0
        
        elif s[1] == 3 and s_prime == s and r == -1:
            return 1.0
        
        else:
            return 0.0
        
    elif a == 'S':
        
        if s == [0, 3]:
            if s_prime == [3, 3] and r == 5:
                return 1.0
            else: 
                return 0.0

        if s[0] < 3 and s_prime == [s[0] + 1,s[1]] and r == 0:
            return 1.0
        
        elif s[0] == 3 and s_prime == s and r == -1:
            return 1.0
        
        else:
            return 0.0
        
    elif a == 'W':

        if s == [0,2]:
            if s_prime == [3,1] and r == 10:
                return 1.0
            else:
                return 0.0
        
        if s[1] > 0 and s_prime == [s[0], s[1] - 1] and r == 0:
            return 1.0
        
        elif s[1] == 0 and s_prime == s and r == -1:
            return 1.0
        
        else:
            return 0.0        

## Policy

In [4]:
def pi(a,s):
    if a == 'N':
        return 0.25
    elif a == 'S':
        return 0.25
    elif a == 'W':
        return 0.25
    else:
        return 0.25        

In [5]:
gamma = 0.9

## Finding the Value Function by finding a fixed point for the Bellman Operator

In [6]:
# Bellman operator
def B(v):
    
    vector = np.array([np.sum([pi(a,s)*prob(s_prime, r, s, a)*(r + gamma * v(s_prime))\
                     for s_prime in states for a in actions for r in rewards]) for s in states])
                         
    def w(s):
        i = states.index(s)
        return vector[i]
    
    return w

In [7]:
def v_0(s):
    return 0.0

In [8]:
eps = 1 
w = v_0
while eps > 1e-6:
    v = B(w)
    eps = (np.array([(v(s) - w(s))**2 for s in states ])).sum()
    w = v

In [9]:
for s in states:
    print(s, v(s))

[0, 0] 4.496253285843853
[0, 1] 9.822826601686613
[0, 2] 4.781709848327758
[0, 3] 3.1890386856796784
[1, 0] 2.2995337042134425
[1, 1] 4.046110308571434
[1, 2] 3.478876211216798
[1, 3] 4.712995696472147
[2, 0] 0.49032610664258397
[2, 1] 1.2907725789796949
[2, 2] 1.3979117254546207
[2, 3] 1.5108641835499377
[3, 0] -0.7893316404350271
[3, 1] -0.19661873005040842
[3, 2] -0.06660972689894851
[3, 3] -0.31865314322959243


## Finding the Value Function by solving system of linear equations

In [10]:
V = np.eye(16)-np.array([np.array([np.sum([pi(a, s)*prob(s_prime, r, s, a)*gamma \
                for a in actions for r in rewards]) for s_prime in states]) for s in states]) 

In [11]:
V

array([[ 0.55 ,  0.   ,  0.   ,  0.   , -0.225,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.225,  0.   ,  0.   ],
       [ 0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.9  ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.775, -0.225,  0.   ,  0.   , -0.225,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.225,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.225,  0.55 ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.225],
       [-0.225,  0.   ,  0.   ,  0.   ,  0.775, -0.225,  0.   ,  0.   ,
        -0.225,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.225,  1.   , -0.225,  0.   ,
         0.   , -0.225,  0.   ,  0.   ,  0.   , -0.225,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.225,  0.   ,  0.   , -0.225,  1.   ,  0.   ,
         0.   ,  0.   , -0.225,  0.   ,  0.   ,  0.   ,  0

In [12]:
C = np.array([np.sum([pi(a, s)*prob(s_prime, r, s, a)*r \
                for a in actions for r in rewards for s_prime in states]) for s in states]).reshape(-1,1)

In [13]:
C

array([[ 2.  ],
       [10.  ],
       [ 2.25],
       [ 0.75],
       [-0.25],
       [ 2.5 ],
       [ 1.25],
       [ 5.  ],
       [-0.25],
       [ 0.  ],
       [ 0.  ],
       [ 1.  ],
       [-0.5 ],
       [-0.25],
       [-0.25],
       [-0.5 ]])

In [14]:
V_inv = np.linalg.inv(V)

In [15]:
V_inv@C

array([[ 4.49841868],
       [ 9.8249919 ],
       [ 4.78387499],
       [ 3.19120373],
       [ 2.3016991 ],
       [ 4.0482756 ],
       [ 3.48104136],
       [ 4.71516071],
       [ 0.4924915 ],
       [ 1.29293787],
       [ 1.40007687],
       [ 1.51302923],
       [-0.78716625],
       [-0.19445344],
       [-0.06444458],
       [-0.3164881 ]])

## State-Action Function

In [16]:
def q(s, a):
       return np.sum([prob(s_prime, r, s, a)*(r + gamma*v(s_prime))\
     for s_prime in states for r in rewards])  

## Generalized Policy Improvement

In [17]:
eps = 1

def v_max(s):
    i = states.index(s)
    return q(s,max_actions[i])
    
    
def pi(a,s):
        i = states.index(s)
        if a == max_actions[i]:
            return 1.0
        else:
            return 0.0
        
    
while eps > 1e-6:    
    max_actions = [actions[np.argmax([q(s, a) for a in actions])] for s in states]

    w = B(v_max)

    eps = np.sum([(v(s) - w(s))**2 for s in states])
    v = w    

In [18]:
([(s, v(s)) for s in states])

[([0, 0], 36.89956987963116),
 ([0, 1], 36.89956987963116),
 ([0, 2], 36.89956987963116),
 ([0, 3], 33.209459448250676),
 ([1, 0], 33.209459448250676),
 ([1, 1], 36.89956987963116),
 ([1, 2], 33.209459448250676),
 ([1, 3], 26.788451064836092),
 ([2, 0], 29.888451064836097),
 ([2, 1], 33.209459448250676),
 ([2, 2], 29.888451064836097),
 ([2, 3], 26.899569879631162),
 ([3, 0], 26.899569879631162),
 ([3, 1], 29.888451064836097),
 ([3, 2], 26.899569879631162),
 ([3, 3], 24.20945944825068)]

In [19]:
[(states[i], max_actions[i]) for i in range(16)]

[([0, 0], 'E'),
 ([0, 1], 'N'),
 ([0, 2], 'W'),
 ([0, 3], 'W'),
 ([1, 0], 'N'),
 ([1, 1], 'N'),
 ([1, 2], 'N'),
 ([1, 3], 'N'),
 ([2, 0], 'N'),
 ([2, 1], 'N'),
 ([2, 2], 'N'),
 ([2, 3], 'W'),
 ([3, 0], 'N'),
 ([3, 1], 'N'),
 ([3, 2], 'N'),
 ([3, 3], 'N')]